# LLMプログラミングの基本

## 準備

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI()

# モデル名
MODEL_NAME = "gpt-5-mini"

## 基本のAPIリクエスト

In [ ]:
# メッセージの設定
message = "言語モデルを使う上でのポイントは"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ]
)

# レスポンスの表示
print(response.model_dump_json(indent=2))

In [ ]:
# 言語モデルからの回答を表示
print(response.choices[0].message.content.strip())

## 役割や前提の設定：`role`

In [ ]:
# 役割や前提の設定
role = "あなたはマーケティング分野に精通したビジネスコンサルタントです。企業の成長をサポートするために、効果的なマーケティング戦略を提供します。"

# メッセージの設定
message = "新製品の発売に向けた効果的なマーケティング戦略を教えてください。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": message},
    ],
)

# 言語モデルからの回答を表示
print(response.choices[0].message.content.strip())

## 出力の多様性：`temperature`、`top_p`

In [ ]:
# モデルの変更
MODEL_NAME = "gpt-4o-mini"

# 出力の多様性
message = "日本の魅力を紹介して！"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ],
    temperature=1.4,
    n=3, # 3回答を得る
    max_completion_tokens=100 # 出力トークン長を制限
)

# 結果を表示（複数回答を比較）
for choice in response.choices:
    print("-" * 20)
    print(choice.message.content.strip())

## 同じ単語の出力抑制：`presence_penalty`、`frequency_penalty`

In [ ]:
# モデルの変更
MODEL_NAME = "gpt-4o-mini"

# 出力の多様性
message = "「a」から始まる5文字以内の英単語を、20個列挙してください。一般的な辞書に載っている英単語のみ。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ],
    presence_penalty=-2.0
)

# 結果を表示（複数回答を比較）
for choice in response.choices:
    print("-" * 20)
    print(choice.message.content.strip())

In [ ]:
# モデルの変更
MODEL_NAME = "gpt-4o-mini"

# 出力の多様性
message = "「a」から始まる5文字以内の英単語を、20個列挙してください。一般的な辞書に載っている英単語のみ。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ],
    presence_penalty=2.0
)

# 結果を表示（複数回答を比較）
for choice in response.choices:
    print("-" * 20)
    print(choice.message.content.strip())

## トークンの選択度合い：`logit_bias`

In [ ]:
# モデルの変更
MODEL_NAME = "gpt-4o-mini"

import tiktoken
from tiktoken.core import Encoding

# OpenAI APIの特定のモデルに対応するトークナイザーを取得
encoding: Encoding = tiktoken.encoding_for_model(MODEL_NAME) # o200k_base

# テキストをトークンIDのリストに変換
tokens = encoding.encode("こんにちは！")
tokens_count = len(tokens)

# トークンの長さとトークンIDを表示
print(f"{tokens_count=}")
print(f"{tokens=}")

In [ ]:
# トークンの選択度合い
message = """
AさんとBさんで会話してください。
A:あ、Bさんだ。
B:
"""

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": message},
    ],
    max_completion_tokens=100,
    n=3,
    logit_bias={95839:8, 3393:8}
)

# 結果を表示
for choice in response.choices:
    print("-" * 20)
    print(choice.message.content.strip())

## 回答をストリーミングで得る：`stream`

In [ ]:
# モデル名
MODEL_NAME = "gpt-5-mini"

# メッセージの設定
message = "言語モデルを使う上でのポイントは"

# APIへリクエスト
stream = client.chat.completions.create(
   model=MODEL_NAME,
   messages=[
       {"role": "user", "content": message},
   ],
   stream=True,
)

for chunk in stream:
   if chunk.choices:
       if chunk.choices[0].delta.content is not None:
           print(chunk.choices[0].delta.content, end="")

## チャットボットの作成


In [ ]:
# メッセージを格納するリスト
messages = []

while True:
   # ユーザーからの質問を受付
   message = input("メッセージを入力:")
   # 質問が入力されなければ終了
   if message.strip()=="":
       break
   print(f"質問:{message}")

   # メッセージにユーザーからの質問を追加
   messages.append({"role": "user", "content": message.strip()})

   # APIへリクエスト
   stream = client.chat.completions.create(
       model=MODEL_NAME,
       messages=messages,
       stream=True,
   )

   # 言語モデルからの回答を表示
   response_message = ""
   for chunk in stream:
       if chunk.choices:
           delta = chunk.choices[0].delta.content
           if delta is not None:
               response_message += delta
               print(delta, end='')
   print()

   # メッセージに言語モデルからの回答を追加
   messages.append({"role": "assistant", "content": response_message})

   # やりとりが8を超えたら古い1往復（user+assistant）から削除
   if len(messages) > 8:
       messages = messages[2:]

print("\n---ご利用ありがとうございました！---")